# Полуаналитическое определение максимального отклонения на основе спектрального разложения матрицы монодромии

Ниже приведена методика определения максимального отклонения с помощью максимизации квадратичной формы с простыми ограничениями методом `L-BFGS-B`:

In [1]:
from utils.libration_sense import *
from scipy.optimize import minimize

In [2]:
orbit_type = 'L1'
orbit_num = 192

# матрица монодромии
M = get_monodromy_matrix(orbit_type, orbit_num)

# начальная точка для интегратора
x0, z0, vy0, T, _, __ = initial_state_parser(orbit_type, orbit_num)
initial_state_da = array.identity(6)
initial_state_da[0] += x0  # x
initial_state_da[2] += z0  # z
initial_state_da[4] += vy0  # v_y
initial_state_cons = initial_state_da.cons()  # положения и скорости центральной точки

# генерируем положения вокруг initial_state из нормального распределения с клиппингом
std_pos = km2du(8)  # 8 км
std_vel = kmS2vu(0.05e-3)  # 0.05 м/c
std_devs = np.array([std_pos] * 3 + [std_vel] * 3)

In [3]:
Mrr = M[:3, :3]
Mrv = M[:3, 3:]
A = np.hstack([Mrr, Mrv])
alpha = 1e-8
ATA = A.T @ A

def f(x, Q):
    result = -np.dot(x, Q @ x)
    return result

def nabla_f(x, Q):
    result = -2 * Q @ x
    return result

bounds = [(-3*std_pos, 3*std_pos)] * 3 + [(-3*std_vel, 3*std_vel)] * 3

In [4]:
np.random.seed(42)
ig = np.random.normal(0, std_devs)
res = minimize(f, ig, args=(ATA,), jac=nabla_f, bounds=bounds, method='L-BFGS-B', options={'disp': True})

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.80722D-08    |proj g|=  1.57834D-04

At iterate    1    f= -1.94892D-06    |proj g|=  4.33921D-05
  ys=-2.590E-06  -gs= 5.857E-07 BFGS update SKIPPED

At iterate    2    f= -1.96596D-06    |proj g|=  0.00000D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6      2      3      8     1     6   0.000D+00  -1.966D-06
  F =  -1.9659612593950037E-006

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [5]:
# Вектор НУ, дающий максимальное отклонение:
opt_initial_state_da = array.identity(6)
opt_initial_state_da[0] += res.x[0] + x0    # x
opt_initial_state_da[1] += res.x[1]         # y
opt_initial_state_da[2] += res.x[2] + z0    # z
opt_initial_state_da[3] += res.x[3]         # vx
opt_initial_state_da[4] += res.x[4] + vy0   # vy
opt_initial_state_da[5] += res.x[5]         # vz

with DA.cache_manager():
    r_f = RK78(opt_initial_state_da, 0.0, T, CR3BP).cons()[:3]

r_0 = initial_state_cons[:3]

print("Максимальное отклонение через виток, L-BFGS-B: ", du2km(np.linalg.norm(r_f - r_0)))

Максимальное отклонение через виток, L-BFGS-B:  538.5548242935496


У нас есть матрица `ATA`, находим её спектральное разложение:

In [6]:
eigvals, S = np.linalg.eigh(ATA)
print(eigvals)
D = np.diag(eigvals)

[-7.19635335e-16 -1.76869825e-16  5.48311425e-15  2.61570948e-01
  3.49607378e+01  6.90156994e+01]


Это вырожденная матрица ранга 3. Формула перехода к диагональному базису:
$$
x=Sy
$$

$$
y=S^Tx
$$

Квадратичная форма $x^T A^TA x$ перейдёт в $y^T D y$, а ограничения $x\in E$ в $y \in S^T E$. Если $E-$прямоугольник фазового пространства, то после ортогонального преобразования он поменяет ориентацию, сохранив свой объём. Отсюда мы перейдём к оптимизационной задаче:
$$
\begin{cases}
y^T D y \longrightarrow \displaystyle{\max_y},\\
y \in S^T E,\\
\end{cases}
$$

Старые ограничения `bounds = [(-3*std_pos, 3*std_pos)] * 3 + [(-3*std_vel, 3*std_vel)] * 3` математически можно выразить как
$$
E = \bigl[-3\sigma_{\text{pos}},\,3\sigma_{\text{pos}}\bigr]^3
    \times
    \bigl[-3\sigma_{\text{vel}},\,3\sigma_{\text{vel}}\bigr]^3.
$$
Новое же множество выглядит так:
$$
S^T E \;=\;\bigl\{\,y\in\mathbb{R}^6 \;\bigm|\;y = S^T x,\;x\in E\bigr\}.
$$

Учитывая только ненулевые компоненты $D$, целевой функционал может быть переписан как
$$
y_1^2d_1^2 + y_2^2d_2^2 + y_3^2d_3^2 \longrightarrow \max_{y_1, y_2, y_3}
$$

От остальных компонент $y_i$, соответствующих нулевым компонентам $d_i$ матрицы $D$, ответ не зависит.

In [7]:
L = np.array([-3*std_pos]*3 + [-3*std_vel]*3)  # shape (6,)
U = np.array([ 3*std_pos]*3 + [ 3*std_vel]*3)  # shape (6,)

# находим индексы ненулевых d_j
J = np.where(np.abs(eigvals) > 1e-14)[0]

# для каждого j∈J считаем границы y_j
y_bounds = {}
for j in J:
    v = S[:, j]  # вектор S_{ij}, i=1..6
    # низ: суммируем S_ij * L_i если S_ij≥0, иначе S_ij*U_i
    y_min = np.sum(np.where(v >= 0, v * L, v * U))
    # верх: суммируем S_ij * U_i если S_ij≥0, иначе S_ij*L_i
    y_max = np.sum(np.where(v >= 0, v * U, v * L))
    y_bounds[j] = (y_min, y_max)

# теперь y_bounds — словарь вида {j:(y_j_min, y_j_max), ...}
print("Активные индексы J:", J)
print("Границы для y_j:", y_bounds)

Активные индексы J: [3 4 5]
Границы для y_j: {np.int64(3): (np.float64(-0.00024819612052096046), np.float64(0.00024819612052096046)), np.int64(4): (np.float64(-0.00018324391043902274), np.float64(0.00018324391043902274)), np.int64(5): (np.float64(-0.00016579028416957844), np.float64(0.00016579028416957844))}


In [8]:
z_bounds = {}
for j in J:
    d = eigvals[j]
    y_min, y_max = y_bounds[j]
    # получаем два конца отрезка в z
    z1 = d * y_min
    z2 = d * y_max
    # правильный нижний и верхний
    z_bounds[j] = (min(z1, z2), max(z1, z2))

print("Bounds for z_j:", z_bounds)

Bounds for z_j: {np.int64(3): (np.float64(-6.492089458435178e-05), np.float64(6.492089458435178e-05)), np.int64(4): (np.float64(-0.0064063423083473995), np.float64(0.0064063423083473995)), np.int64(5): (np.float64(-0.011442132416470137), np.float64(0.011442132416470137))}


In [9]:
import itertools

# перечисляем для каждого j в J кортеж (min,max)
intervals_y = [y_bounds[j] for j in J]  # [(y3_min,y3_max), (y4_min,y4_max), (y5_min,y5_max)]

# получаем 2^3 = 8 комбинаций концов
corners_y = list(itertools.product(*intervals_y))
# corners — список из 8 кортежей вида (y3_corner, y4_corner, y5_corner)

# если вам нужно собрать их в полном 6-мерном y-векторе,
# можно «заполнить» остальные позиции нулями (или чем угодно):
vertices_y = []
for corner in corners_y:
    y_vec = np.zeros(6)
    for idx, val in zip(J, corner):
        y_vec[idx] = val
    vertices_y.append(y_vec)

vertices_z = np.array(vertices_y)
print(len(corners_y))

8


In [10]:
# перечисляем для каждого j в J кортеж (min,max)
intervals_z = [z_bounds[j] for j in J]  # [(y3_min,y3_max), (y4_min,y4_max), (y5_min,y5_max)]

# получаем 2^3 = 8 комбинаций концов
corners_z = list(itertools.product(*intervals_z))
# corners — список из 8 кортежей вида (y3_corner, y4_corner, y5_corner)

# если вам нужно собрать их в полном 6-мерном y-векторе,
# можно «заполнить» остальные позиции нулями (или чем угодно):
vertices_z = []
for corner in corners_z:
    y_vec = np.zeros(6)
    for idx, val in zip(J, corner):
        y_vec[idx] = val
    vertices_z.append(y_vec)

vertices_z = np.array(vertices_z)
print(len(corners_z))

8


In [11]:
for corner in corners_z:
    z_vec = np.array(corner)
    print(z_vec.T @ z_vec)

0.00017196783073031223
0.00017196783073031223
0.00017196783073031223
0.00017196783073031223
0.00017196783073031223
0.00017196783073031223
0.00017196783073031223
0.00017196783073031223


In [12]:
for corner in corners_y:
    y_vec = np.array(corner)
    print(y_vec.T @ y_vec)

1.226660632796693e-07
1.226660632796693e-07
1.226660632796693e-07
1.226660632796693e-07
1.226660632796693e-07
1.226660632796693e-07
1.226660632796693e-07
1.226660632796693e-07


In [13]:
z_bounds_standart = [value for value in z_bounds.values()]
print(z_bounds_standart)

[(np.float64(-6.492089458435178e-05), np.float64(6.492089458435178e-05)), (np.float64(-0.0064063423083473995), np.float64(0.0064063423083473995)), (np.float64(-0.011442132416470137), np.float64(0.011442132416470137))]


In [14]:
from itertools import product

# строим все комбинации (cartesian product)
vertices = np.array(list(product(*z_bounds_standart)))

print(vertices)
print(vertices.shape)

[[-6.49208946e-05 -6.40634231e-03 -1.14421324e-02]
 [-6.49208946e-05 -6.40634231e-03  1.14421324e-02]
 [-6.49208946e-05  6.40634231e-03 -1.14421324e-02]
 [-6.49208946e-05  6.40634231e-03  1.14421324e-02]
 [ 6.49208946e-05 -6.40634231e-03 -1.14421324e-02]
 [ 6.49208946e-05 -6.40634231e-03  1.14421324e-02]
 [ 6.49208946e-05  6.40634231e-03 -1.14421324e-02]
 [ 6.49208946e-05  6.40634231e-03  1.14421324e-02]]
(8, 3)


In [15]:
for vertice in vertices:
    print(vertice.T @ vertice)

0.00017196783073031223
0.00017196783073031223
0.00017196783073031223
0.00017196783073031223
0.00017196783073031223
0.00017196783073031223
0.00017196783073031223
0.00017196783073031223


In [16]:
ig = np.random.randn(vertices.shape[1])
res = minimize(f, ig, args=(np.eye(vertices.shape[1]),), jac=nabla_f, bounds=z_bounds_standart, method='L-BFGS-B', options={'disp': True})

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         3 variables are exactly at the bounds

At iterate    0    f= -1.71968D-04    |proj g|=  0.00000D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      0      1      0     0     0   0.000D+00  -1.720D-04
  F =  -1.7196783073031223E-004

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [17]:
 res.x

array([ 6.49208946e-05,  6.40634231e-03, -1.14421324e-02])

In [18]:
res

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: -0.00017196783073031223
        x: [ 6.492e-05  6.406e-03 -1.144e-02]
      nit: 0
      jac: [-1.298e-04 -1.281e-02  2.288e-02]
     nfev: 1
     njev: 1
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>

In [19]:
z_opt = res.x
y_opt = np.zeros(6)
y_opt[J] = z_opt / np.nonzero(eigvals > 1e-10)[0]
print(y_opt)

x_opt = S @ y_opt
print(x_opt)

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.16402982e-05
  1.60158558e-03 -2.28842648e-03]
[ 0.00194851 -0.00035618  0.00139783  0.00013014  0.00136927 -0.00018203]


In [20]:
# Вектор НУ, дающий максимальное отклонение:
opt_initial_state_da = array.identity(6)
opt_initial_state_da[0] += x_opt[0] + x0    # x
opt_initial_state_da[1] += x_opt[1]         # y
opt_initial_state_da[2] += x_opt[2] + z0    # z
opt_initial_state_da[3] += x_opt[3]         # vx
opt_initial_state_da[4] += x_opt[4] + vy0   # vy
opt_initial_state_da[5] += x_opt[5]         # vz

with DA.cache_manager():
    r_f = RK78(opt_initial_state_da, 0.0, T, CR3BP).cons()[:3]

r_0 = initial_state_cons[:3]

print("Максимальное отклонение через виток, полуаналитика: ", du2km(np.linalg.norm(r_f - r_0)))

Максимальное отклонение через виток, полуаналитика:  8504.118348068172


In [22]:
print(std_pos)
print(std_vel)
print(bounds)
print(x_opt)

2.0811383826953342e-05
4.880238020495532e-05
[(-6.243415148086002e-05, 6.243415148086002e-05), (-6.243415148086002e-05, 6.243415148086002e-05), (-6.243415148086002e-05, 6.243415148086002e-05), (-0.00014640714061486595, 0.00014640714061486595), (-0.00014640714061486595, 0.00014640714061486595), (-0.00014640714061486595, 0.00014640714061486595)]
[ 0.00194851 -0.00035618  0.00139783  0.00013014  0.00136927 -0.00018203]


In [23]:
from scipy.spatial import ConvexHull          # только чтобы отфильтровать лишнее

def x_corners(std_pos, std_vel):
    lows  = np.array([-3*std_pos]*3 + [-3*std_vel]*3)
    highs = -lows
    return np.array([np.where(mask, highs, lows)
                     for mask in product([0, 1], repeat=6)])

def z_vertices(S, D, std_pos, std_vel, eps=1e-12):
    """Вернёт массив уникальных вершин зонатопа W·X (всегда shape (N,3))."""
    W   = D @ S.T
    Z64 = (W @ x_corners(std_pos, std_vel).T).T          # (64,3)
    Z   = np.unique(np.round(Z64, 12), axis=0)           # убираем дубликаты

    # --- определяем реально изменяющиеся координаты ------------------------
    var   = Z.var(axis=0)
    dims  = np.where(var > eps)[0]                       # столбцы, где есть разброс
    k     = len(dims)                                    # фактическая размерность

    if k == 0:                                           # все точки совпали
        return Z[:1]

    if k == 1:
        d = dims[0]
        idx_min = np.argmin(Z[:, d])
        idx_max = np.argmax(Z[:, d])
        return Z[[idx_min, idx_max]]

    # k == 2  или 3: строим ВСУ оболочку в меньшей размерности
    Z_proj = Z[:, dims]
    hull   = ConvexHull(Z_proj) if len(Z) > k else None
    verts  = Z[hull.vertices] if hull else Z
    return verts


V = z_vertices(S, D, std_pos, std_vel)
print(f"Количество вершин: {len(V)}")
print(V)

Количество вершин: 32
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.49208950e-05
  -1.96672880e-03  5.10334930e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.27417730e-05
  -1.95143476e-03 -2.20787692e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -5.41062660e-05
  -5.61597936e-03  5.35257006e-03]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -5.29031320e-05
   1.90065621e-03 -1.61531089e-03]
 [ 0.00000000e+00  0.00000000e+00 -0.00000000e+00 -5.24898470e-05
  -2.74179771e-03 -4.61556722e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -5.19271450e-05
  -5.60068532e-03 -1.95865615e-03]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -5.07240110e-05
   1.91595025e-03 -8.92653711e-03]
 [ 0.00000000e+00  0.00000000e+00 -0.00000000e+00 -4.38543400e-05
  -6.40634231e-03  2.94487976e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.16752190e-05
  -6.39104827e-03 -4.36634645e-03]
 [ 0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.04720840e-05
   

In [24]:
max_vert_val = -9e16
max_vert = None
for vertice in V:
    temp = vertice.T @ vertice
    if temp > max_vert_val:
        max_vert_val = temp
        max_vert = vertice

print(max_vert)

[ 0.00000000e+00 -0.00000000e+00  0.00000000e+00 -5.78481400e-06
 -2.12746359e-03  1.14421324e-02]


In [26]:
from scipy.linalg import null_space   # pinv = псевдообратная Мура-Пенроуза

from numpy.linalg import pinv          # ← берём из NumPy, у него всегда rcond


def x_from_z(z, S, D, return_nullspace=False, rtol=1e-12):
    """
    Возвращает:
      x_star          – минимальное по норме решение W x = z
      (опц.) N_basis  – базис ядра W, если нужен полный набор решений
    """
    W = D @ S.T                 # (3,6) – фактически ненулевые строки
    W_pinv = pinv(W, rcond=1e-14)
    x_star = W_pinv @ z         # минимальная норма

    if return_nullspace:
        N = null_space(W, rcond=rtol)   # (6,3) – столбцы образуют базис
        return x_star, N

    return x_star

x_opt = x_from_z(max_vert, S, D, return_nullspace=False)
x_opt

array([-1.42341756e-04,  4.22434064e-05, -4.83407331e-05, -1.52681353e-05,
       -8.15784523e-05,  2.01973622e-05])

In [27]:
# Вектор НУ, дающий максимальное отклонение:
opt_initial_state_da = array.identity(6)
opt_initial_state_da[0] += x_opt[0] + x0    # x
opt_initial_state_da[1] += x_opt[1]         # y
opt_initial_state_da[2] += x_opt[2] + z0    # z
opt_initial_state_da[3] += x_opt[3]         # vx
opt_initial_state_da[4] += x_opt[4] + vy0   # vy
opt_initial_state_da[5] += x_opt[5]         # vz

with DA.cache_manager():
    r_f = RK78(opt_initial_state_da, 0.0, T, CR3BP).cons()[:3]

r_0 = initial_state_cons[:3]

print("Максимальное отклонение через виток, полуаналитика: ", du2km(np.linalg.norm(r_f - r_0)))

Максимальное отклонение через виток, полуаналитика:  546.1322542171926


In [29]:
def in_bounds(x, std_pos, std_vel, atol=1e-12, verbose=True):
    """
    Проверяет x ∈ [-3σ_pos, 3σ_pos]^3 × [-3σ_vel, 3σ_vel]^3.

    Параметры
    ---------
    x        : np.ndarray shape (6,) — проверяемый вектор
    std_pos  : float — σ для координат позиций (первые три)
    std_vel  : float — σ для координат скоростей (последние три)
    atol     : float — допускаемая погрешность (|x|-граница)
    verbose  : bool  — печатать подробности

    Возвращает
    ----------
    ok : bool — True, если все компоненты удовлетворяют ограничениям
    """
    lower = np.array([-3*std_pos]*3 + [-3*std_vel]*3)
    upper = -lower                                           # симметрично

    below_lower = x < lower - atol
    above_upper = x > upper + atol
    ok = not (below_lower | above_upper).any()

    if verbose:
        if ok:
            print("✓ x_opt удовлетворяет всем ограничениям.")
        else:
            bad_idx = np.where(below_lower | above_upper)[0]
            for i in bad_idx:
                print(f"× компонентa {i}: {x[i]:.6g} "
                      f"вне [{lower[i]:.6g}, {upper[i]:.6g}]")
    return ok

in_bounds(x_opt, std_pos, std_vel)

× компонентa 0: -0.000142342 вне [-6.24342e-05, 6.24342e-05]


False